In [10]:
%load_ext autoreload
import pandas as pd
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import glob
from tqdm.notebook import tqdm
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%autoreload 1

In [3]:
path = '../ignored/tvsum50_ver_1_1/ydata-tvsum50-v1_1/data/ydata-tvsum50-anno.tsv'
df = pd.read_csv(path,delimiter='\t', header=None, names=['id','cat','scores'])
df = df.join(df['scores'].str.split(',',expand=True).apply(pd.to_numeric))
df = df.drop(axis=1, columns=['scores'])

In [11]:
import sys
sys.path.append('../src')
%aimport evaluation

In [40]:
files = []
for file_id in glob.glob('../inputs/*'):
    filename = file_id.split('/')[-1].split('.')[0]
    if filename not in files:
        files.append(filename)
order = [ "AwmHb44_ouw", "98MoyGZKHXc", "J0nA4VgnoCo", "gzDbaEs1Rlg", "XzYM3PfTM4w", "HT5vyqe0Xaw", "sTEELN-vY30", "vdmoEJ5YbrQ", "xwqBXPGE9pQ", "akI8YFjEmUw", "i3wAGJaaktw", "Bhxk-O1Y7Ho", "0tmA_C6XwfM", "3eYKfiOEJNs", "xxdtq8mxegs", "WG0MBPpPC6I", "Hl-__g2gn_A", "Yi4Ij2NM7U4", "37rzWOQsNIw", "LRw_obCPUt0", "cjibtmSLxQ4", "b626MiF1ew4", "XkqCExn6_Us", "GsAD1KT1xo8", "PJrm840pAUI", "91IHQYk1IQM", "RBCABdttQmI", "z_6gVvQb2d0", "fWutDQy1nnY", "4wU_LUjG5Ic", "VuWGsYPqAX8", "JKpqYvAdIsw", "xmEERLqJ2kU", "byxOvuiIJV0", "_xMr-HKMfVA", "WxtbjNsCQ8A", "uGu_10sucQo", "EE-bNr36nyA", "Se3oxnaPsz0", "oDXZc0tZe04", "qqR6AEXwxoQ", "EYqVtI9YWJA", "eQu1rNs0an0", "JgHubY5Vw3Y", "iVt07TCkFM0", "E11zDS9XGzg", "NyBmCxDoHJU", "kLxoNp-UchI", "jcoYJXDG9sw", "-esJrBWj2d8"]

files = sorted(files, key={k:v for v,k in enumerate(order)}.get)

In [90]:
all_val_list = []
clip_fps = []
for file_id in tqdm(order):
    frame_scores_for_video = df[df['id']==file_id].drop(axis=1, columns=['id','cat']).values
    frame_scores_for_video_nonan = frame_scores_for_video[~np.isnan(frame_scores_for_video)]
    frame_scores_for_video_nonan = frame_scores_for_video_nonan.reshape((20,-1))
    temp = []
    if file_id in files:
        with open(f'../intermediate/{file_id}_clip_list.json') as f:
            clip_data = json.load(f)

        # Convert clip start time and end time to fps
        fps = round(cv2.VideoCapture(f'../inputs/{file_id}.mp4').get(cv2.CAP_PROP_FPS))
        for clip in clip_data:
            h,m,s = clip['clip_start'].split(':')
            secs = int(h)*3600+int(m)*60+float(s)
            clip['clip_start_fps'] = round(secs*fps)
            h,m,s = clip['clip_end'].split(':')
            secs = int(h)*3600+int(m)*60+float(s)
            clip['clip_end_fps'] = round(secs*fps)
            temp.append([clip['clip_start_fps'], clip['clip_end_fps']])
        # Generate list with included and excluded frames. Included frames are 1.
        val_list = [0 for i in range(frame_scores_for_video_nonan.shape[1])]
        try:
            for clip in clip_data:
                for i in range(clip['clip_start_fps'], clip['clip_end_fps']+1):
                    val_list[i] = 1
            val_np = np.array(val_list)
        except:
            pass
        
    elif file_id in order:
        print(file_id)
        val_list = [0 for i in range(frame_scores_for_video_nonan.shape[1])]
        temp.append([])
    all_val_list.append(val_list)
    clip_fps.append(temp)
#     for clip in clip_data:
#         print(clip['clip_start_fps'], clip['clip_end_fps'])

#     # Plot the average score graph
#     plt.plot(frame_scores_for_video_nonan.mean(axis=0))
    
#     # Plot the regions of the selected frames
#     for i in range(len(val_list)):
#         if val_list[i] == 1:
#             plt.axvspan(i,i+1,facecolor='b', alpha=0.5)

#     # Plot labels and axes setup
#     plt.xlabel('Frame Number')
#     plt.ylabel('Average Score over 20 evaluations')
#     red_patch = mpatches.Patch(color='blue', alpha=0.5, label='Selected in Summary')
#     plt.legend(handles=[red_patch])

#     # Create directories and save
#     if not os.path.exists(f"../outputs/{evaluation.proportion}_{evaluation.flexibility}/"):
#         os.makedirs(f"../outputs/{evaluation.proportion}_{evaluation.flexibility}/")
#     print(f"{evaluation.proportion}_{evaluation.flexibility}/{file_id}")
#     plt.savefig(f"../outputs/{evaluation.proportion}_{evaluation.flexibility}/{file_id}.png", dpi=300)
#     plt.clf()

gzDbaEs1Rlg
sTEELN-vY30
xxdtq8mxegs
LRw_obCPUt0
cjibtmSLxQ4
XkqCExn6_Us
GsAD1KT1xo8
PJrm840pAUI
91IHQYk1IQM
RBCABdttQmI
z_6gVvQb2d0
fWutDQy1nnY
VuWGsYPqAX8
JKpqYvAdIsw
xmEERLqJ2kU
byxOvuiIJV0
_xMr-HKMfVA
uGu_10sucQo
oDXZc0tZe04
NyBmCxDoHJU
kLxoNp-UchI
-esJrBWj2d8



In [91]:
clip_fps_data = np.array([np.array(i) for i in clip_fps])


In [93]:
from scipy.io import savemat
array = np.array([np.array(i, dtype=float) for i in all_val_list])
savemat('shots.mat', {"shot_boun": clip_fps_data})

array([0., 0., 0., ..., 0., 0., 0.])

In [77]:
for idx,i in enumerate(order):
    print(idx+1,i)

1 AwmHb44_ouw
2 98MoyGZKHXc
3 J0nA4VgnoCo
4 gzDbaEs1Rlg
5 XzYM3PfTM4w
6 HT5vyqe0Xaw
7 sTEELN-vY30
8 vdmoEJ5YbrQ
9 xwqBXPGE9pQ
10 akI8YFjEmUw
11 i3wAGJaaktw
12 Bhxk-O1Y7Ho
13 0tmA_C6XwfM
14 3eYKfiOEJNs
15 xxdtq8mxegs
16 WG0MBPpPC6I
17 Hl-__g2gn_A
18 Yi4Ij2NM7U4
19 37rzWOQsNIw
20 LRw_obCPUt0
21 cjibtmSLxQ4
22 b626MiF1ew4
23 XkqCExn6_Us
24 GsAD1KT1xo8
25 PJrm840pAUI
26 91IHQYk1IQM
27 RBCABdttQmI
28 z_6gVvQb2d0
29 fWutDQy1nnY
30 4wU_LUjG5Ic
31 VuWGsYPqAX8
32 JKpqYvAdIsw
33 xmEERLqJ2kU
34 byxOvuiIJV0
35 _xMr-HKMfVA
36 WxtbjNsCQ8A
37 uGu_10sucQo
38 EE-bNr36nyA
39 Se3oxnaPsz0
40 oDXZc0tZe04
41 qqR6AEXwxoQ
42 EYqVtI9YWJA
43 eQu1rNs0an0
44 JgHubY5Vw3Y
45 iVt07TCkFM0
46 E11zDS9XGzg
47 NyBmCxDoHJU
48 kLxoNp-UchI
49 jcoYJXDG9sw
50 -esJrBWj2d8


In [81]:
files
print('[')
for i in files:
    print(f"\"{i}\"", end=' ')
print(']')

[
"AwmHb44_ouw" "98MoyGZKHXc" "J0nA4VgnoCo" "XzYM3PfTM4w" "HT5vyqe0Xaw" "vdmoEJ5YbrQ" "xwqBXPGE9pQ" "akI8YFjEmUw" "i3wAGJaaktw" "Bhxk-O1Y7Ho" "0tmA_C6XwfM" "3eYKfiOEJNs" "WG0MBPpPC6I" "Hl-__g2gn_A" "Yi4Ij2NM7U4" "37rzWOQsNIw" "b626MiF1ew4" "4wU_LUjG5Ic" "WxtbjNsCQ8A" "EE-bNr36nyA" "Se3oxnaPsz0" "qqR6AEXwxoQ" "EYqVtI9YWJA" "eQu1rNs0an0" "JgHubY5Vw3Y" "iVt07TCkFM0" "E11zDS9XGzg" "jcoYJXDG9sw" ]
